In [ ]:
import numpy as np
import pandas as pd
import scipy.signal
from scipy.stats import logistic
import matplotlib.pyplot as plt
from datetime import timedelta
from scipy.integrate import simpson
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from fdasrsf.regression import elastic_regression
from fdasrsf.utility_functions import elastic_distance
import fdasrsf as fs
import seaborn as sns
import tsfel
import plotly.express as px
import plotly.graph_objects as go
import pingouin as pg

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install fdasrsf

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Data loading

In [ ]:
# baseline_df = pd.read_excel('Combined_DATA_11.12.21s.xlsx', engine='openpyxl')
baseline_df = pd.read_csv('final_clinical_df.csv')
baseline_df = baseline_df.drop(columns=['Dom','Gender'])#.astype('float')

# ys = np.loadtxt('ys_s.txt')
# pids = np.loadtxt('pids_s.txt')
# visits = np.loadtxt('visits_s.txt', dtype=str)

ysp = np.load('ys_curl_new.npy')[:,:,0]
pidsp = np.loadtxt('pids_curl_new.txt')
visitsp = np.loadtxt('visits_curl_new.txt', dtype=str)

# ysp = np.load('all_knock.npy')[:,:,1]
# pidsp = np.loadtxt('pids_knock.txt')
# visitsp = np.loadtxt('visits_knock.txt', dtype=str)

df_processor = pd.DataFrame(ysp)
df_processor['pids']=pidsp
df_processor['visits']=visitsp

baseline_df['Normalized_Dynamometry (N/cm)'] = baseline_df['Avg. Dynamometry (N)'] / baseline_df['Forearm_Length (cm)']
baseline_df['Normalized Elbow Torque (Nm/cm)'] = baseline_df['Elbow Torque (Nm)'] / baseline_df['Forearm_Length (cm)']

df_processor['visits'] = pd.to_datetime(df_processor['visits'])
df_processor['visits'] = df_processor['visits'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_processor['visits'] = pd.to_datetime(df_processor['visits'], format = "%Y-%m-%d")

baseline_df['Sensor Data Date'] = pd.to_datetime(baseline_df['Sensor Data Date'], format = "%Y-%m-%d")
df_processor = pd.merge(df_processor, baseline_df, left_on=['pids','visits'], right_on=['PID','Sensor Data Date'], how='inner')
# df_processor = pd.merge(df_processor, baseline_df, left_on=['pids'], right_on=['PID'], how='inner')

# df_processor['visits'] = pd.to_datetime(df_processor['visits'])
# baseline_df['Sensor Data Date'] = pd.to_datetime(baseline_df['Sensor Data Date'])

# df_processor = pd.merge(df_processor, baseline_df, left_on=['pids','visits'], right_on=['PID','Sensor Data Date'], how='inner')

In [ ]:
# Function to randomly select 1 row from every 2 rows for each group
# def select_random_row(group):
#     if len(group) > 1:
#         return group.sample(n=1, random_state=42)
#     return group

# # Group by 'pids', then apply the selection function, and reset index
# df_processor = df_processor.groupby('pids', group_keys=False).apply(lambda x: x.groupby(np.arange(len(x)) // 2).apply(select_random_row)).reset_index(drop=True)

# df_processor['pids']

## Alignment

In [ ]:
ys = df_processor.iloc[:,0:400].values

time = np.array(range(400)).astype(float)
obj = fs.fdawarp(np.transpose(ys), time)

""" Calculate healthy mean"""
obj.srsf_align(parallel=True)

In [ ]:
# data = np.load('MPEG7/Xdata.npy',allow_pickle=True)
# Xdata = data
# curve = Xdata[0,1]
# n,M = curve.shape
# K = Xdata.shape[1]

# n = 2
# M = 400
# K = ys.shape[0]

# beta = np.zeros((n,M,K))
# for i in range(0,K):
#     beta[0,:,i] = np.arange(0, 400, 1, dtype=int)
#     beta[1,:,i] = ys[i, :]

In [ ]:
# print(beta.shape, ys.shape)

In [ ]:
# beta = ys

In [ ]:
# for i in range(beta.shape[2]):
#     plt.figure()
#     plt.plot(beta[0,:,i], beta[1,:,i])
#     plt.show()

In [ ]:
#obj2 = fs.fdacurve(beta, mode='O', N=M)

In [ ]:
#obj2.karcher_mean()

In [ ]:
#obj2.srvf_align(rotation=False)

In [ ]:
#obj2.plot()

In [ ]:
#obj2.karcher_cov()
#obj2.shape_pca()

In [ ]:
#obj2.plot_pca()

## FPCA

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30, 3))
vpca = fs.fdavpca(obj)
vpca.calc_fpca(no=3)
vpca.plot()

vpca_df = pd.DataFrame(vpca.coef, columns=['vpc1', 'vpc2', 'vpc3'])

hpca = fs.fdahpca(obj)
hpca.calc_fpca(no=3)

hpca.plot()

hpca_df = pd.DataFrame(hpca.coef, columns=['hpc1', 'hpc2', 'hpc3'])

jpca = fs.fdajpca(obj)
jpca.calc_fpca(no=3)

jpca.plot()

jpca_df = pd.DataFrame(jpca.coef, columns=['jpc1', 'jpc2', 'jpc3'])

pca_df = pd.concat([hpca_df, vpca_df, jpca_df], axis=1)

#pca_df
df_pca_analyze = pd.concat([df_processor.iloc[:,401:,], pca_df], axis=1)

In [ ]:
cumm_coef = 100 * (vpca.latent) / sum(vpca.latent)

sns.set_style(style='white')
fig, axs = plt.subplots(1, 3, figsize=(15, 3))
for i, ax in enumerate(axs):
    ax.plot(vpca.f_pca[:,2,i], label=r'$\mu + 1 \sigma v$', color='tab:red')
    ax.plot(vpca.f_pca[:,1,i], label=r'$\mu$', color='black')
    ax.plot(vpca.f_pca[:,0,i], label=r'$\mu - 1 \sigma v$', color='tab:blue')
    ax.set_title("VPC{}, {}%".format(i+1, round(cumm_coef[i], 0)))
    ax.legend()
plt.savefig('vertical_eigenfunctions_long.png', dpi=300, bbox_inches='tight')

In [ ]:
df_long = df_pca_analyze.groupby(['PID','Visit #'], as_index=False).agg({'Age':'mean',
                                    'Forearm_Length (cm)':'mean',
                                    'CSA (cm^2)':'mean',
                                    'Avg_Echo':'mean',
                                    'Normalized Elbow Torque (Nm/cm)':'mean',
                                    'vpc1':'mean',
                                    'vpc2':'mean',
                                    'vpc3':'mean',
                                    'Brooke':'mean',
                                    'Cohort':'first',
                                    'PID':'first','visits':'first'})

#df_long = df_pca_analyze

df_long.to_csv('df_long_mean.csv', index=False)

In [ ]:
df_long

In [ ]:
# from statannot import add_stat_annotation

# feat_cols = ['Normalized Elbow Torque (Nm/cm)','Avg_Echo','vpc1','vpc3']
# fig, axs = plt.subplots(1, 4, figsize=(3*len(feat_cols), 4))


# for i, feat_col in enumerate(feat_cols):

#     pids_df_subset = df_long[[feat_col, 'Cohort']]
#     a = sns.boxplot(data=pids_df_subset, x='Cohort', y=feat_col, ax=axs[i])

    # add_stat_annotation(a, data=pids_df_subset, x='Cohort', y=feat_col, 
    #                     box_pairs=[("DMD", "Healthy"), ("DMD", "SMA"), ("SMA", "Healthy")],
    #                     test='t-test_ind', text_format='simple')

## Correlation

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sb

sns.set(font_scale=1.2)
plt.figure(figsize = (16,2))


# feat_columns = ['PID','Age', 'Forearm_Length (cm)','Location of dynamometry from wrist (cm)', 'Avg. Dynamometry (N)',
#         'Normalized_Dynamometry (N/cm)','Elbow Torque (Nm)','Normalized Elbow Torque (Nm/cm)',
#        'CSA (cm^2)', 'Normalized_CSA (cm^2/cm)',
#        'Avg_Long_Echo', 'Avg_Trans_Echo', 'Avg_Echo', 'Norm_Avg_Echo',
#        'Est_Specific_Tension', 'Est_Specific_Tension_normCSA','Est_Specific_Tension', 'Est_Specific_Tension_normCSA',
#        'Muscle_score1', 'Muscle Score2', 'Muscle_score3', 'Muscle_score4',
#        'Brooke','hpc1', 'hpc2', 'hpc3', 'vpc1', 'vpc2',
#        'vpc3']
#

feat_columns = ['PID','Age','CSA (cm^2)','Normalized Elbow Torque (Nm/cm)','Avg_Echo','Brooke',
                'vpc1','vpc2','vpc3']

# df = df_pca_analyze.groupby(['PID']).agg({'Age':'mean',
#                                     'Forearm_Length (cm)':'mean',
#                                     'CSA (cm^2)':'mean',
#                                     'Avg_Echo':'mean',
#                                     'Normalized Elbow Torque (Nm/cm)':'mean',
#                                     'vpc1':'mean',
#                                     'vpc2':'mean',
#                                     'vpc3':'mean',
#                                     'Brooke':'mean',
#                                     'Cohort':'first',
#                                     'PID':'first'})

df = df_long

clist = ['DMD','SMA','Healthy']
corrs = []

for c in clist:

    df_b = df[df['Cohort'] == c]
    #print(df_b.shape)
    corr = df_b[feat_columns].corr().loc['vpc1'].iloc[1:-3]

    #p_values = corr_sig1(df)              
    #mask = np.invert(np.tril(p_values < 1))
    #selected_mask = mask[8:-1,:-7]

    corrs.append(corr)

    #dataplot = sb.heatmap(corr3, cmap=sb.diverging_palette(0, 255, sep=77, as_cmap=True), #mask=selected_mask,
                          #annot=True, vmin=-1, vmax=1, xticklabels = True)

    #displaying heatmap
    # plt.title('Correlations for Healthy')

    # plt.savefig('corr_Healthy.png',bbox_inches='tight',dpi=300)
    #plt.show()
    
    
df_corr = pd.concat(corrs, axis=1)
df_corr.columns = ['DMD','SMA','Healthy']

df_corr = df_corr.transpose()
df_corr = df_corr.rename(columns={'Normalized Elbow Torque (Nm/cm)': 'NET (Nm/cm)',
                                  'Avg_Echo':'Avg_Echo (gsv)'})

sns.set(font_scale=1.2)
plt.figure(figsize = (17,3))

dataplot = sb.heatmap(df_corr, cmap=sb.diverging_palette(0, 255, sep=77, as_cmap=True), 
                      annot=True, vmin=-1, vmax=1, xticklabels = True)

  
#displaying heatmap
#plt.title('Cohort Correlations for VPC3 (Curl)')

#plt.savefig('corr_vpc3curl_random.png',bbox_inches='tight',dpi=300)
plt.show()

In [ ]:
#df.to_csv('curl_vpc.csv',index=False)

In [ ]:
df_pca_analyze[df_pca_analyze['PID'] == 11]
#df_pca_analyze.sort_values(by=['vpc1'])[['PID','vpc1','visits','Visit #']]                                    

In [ ]:
#df_b = df[(df['Cohort'] == 'DMD') | (df['Cohort'] == 'SMA')]
#df_b

In [ ]:
# ft = ['vpc1','Forearm_Length (cm)']
# df_b[ft].corr()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from scipy import stats

# # Sample data creation

# x = df_b['vpc1'].values
# y = df_b['Forearm_Length (cm)'].values

# # Calculating the correlation coefficient
# #correlation_coefficient, _ = stats.pearsonr(x, y)
# correlation_coefficient = 0.61

# # Scatterplot
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x=x, y=y)  # Adjusted to use keyword arguments

# # Adding the correlation line
# sns.regplot(x=x, y=y, scatter=False, color="green")

# # Annotating the plot with the correlation coefficient
# plt.text(max(x)*0.55, max(y)*0.5, f'Correlation: {correlation_coefficient:.2f}', fontsize=12)

# plt.xlabel('Wearable Measures')
# plt.ylabel('Clinical Measures')
# plt.savefig('cca_corr.png', dpi=300)
